# Algorithm
![DDPG Algorithm](pic/DDPG.jpg)

## 算法核心思想
1.对于连续控制问题不太可能离散使用DQN来解决，直接用确定策略网络（deterministic policy network）生成一个确定的梯度 <br>
2.其可以看作以DQN为基础，actor-critic架构辅助更新的算法<br>
3.与DQN类似，考虑到同时更新critic和actor将会导致bootstrapping现象导致高估（overestimation） critic网络，对actor和critic网络使用target-network方法，在TD更新critic网络时替代$r_t+\gamma q_{t+1}$，此处还能用multi-step TD targets的方法进一步提高critic网络的更新参数。参见上式第12行<br>
4.更新完critic网络后，再更新actor网络，公式为$$\nabla_\theta \pi(s_t;\theta)\times meanOF(\nabla_\pi Q(s_t,\pi(s_t;\theta);\omega))$$
5.此处还能用experience reply，建立一个足够大的（1e5）的reply buffer

## 算法提升
1.TD3<br>
1.1学习double DQN的方法，用两个critic网络对动作值进行评估，以解决（fix）高估问题<br>
1.2延迟更新,actor网络的更新频率比critic网络慢<br>
1.3“梯度截取，这是在知乎中有人提到的，但是[文章](https://zhuanlan.zhihu.com/p/553179826)中并没有体现出来<br>
1.4目标策略平滑，首先在探索时加入噪声（并clip一下），在生成target时加入噪声
$$y=r+\gamma Q_{\theta^{\prime}}\left(s^{\prime}, \pi_{\omega^{\prime}}\left(s^{\prime}\right)+\epsilon\right)$$
$$\epsilon \sim \operatorname{clip}(\mathcal{N}(0, \sigma),-c, c)$$
但是考虑到误差问题，再次clip，最终为
$$y=r+\gamma Q_{\theta^{\prime}}\left(s^{\prime}, \operatorname{clip}\left(\pi_{\omega^{\prime}}\left(s^{\prime}\right)+\epsilon\right.\right., min action, max action \left.)\right), \epsilon \sim \operatorname{clip}(\mathcal{N}(0, \sigma),-c, c)$$

## 总流程
首先初始化四个网络参数<br>
<1>actor 网络参数$\theta$<br>
<2>actor_target_network参数$\theta^{\prime}\gets \theta$<br>
<3>critic 网络参数$\omega$<br>
<4>critic_target_network 参数$\omega^{\prime}\gets \omega$<br>

## 进入循环<br>
观测状态s,并且执行动作a，a为clip（actor网络的值加上一个正态分布的噪声）<br>
执行a，得到下一时刻的状态s',reward r，和完成与否的flag d<br>
将上述的transition $(s,a,r,s')$ 存到replay buffer里<br>
如果d=1，即本次探索结束了，就从头开始reset()<br>
重复上述操作，做到充分探索，直到replay buffer满了

如果replay buffer满了，就开始更新网络参数<br>
在一定次数的循环中
采取小批量更新，从经验池中随机选取一定批量的transition
计算targets
$$y\left(r, s^{\prime}, d\right)=r+\gamma(1-d) Q_{\omega^{\prime}}\left(s^{\prime}, \pi_{\theta^{\prime}}\left(s^{\prime}\right)\right)$$

上式中 $(1-d)$ 表示是否完成，如果没有完成肯定是要加入Q的值，如果没有就不用加<br>
此外，很明显使用了target network来辅助计算

用TD算法 gradient descent 更新一次critic网络的值<br>
$$\nabla_{\omega}meanOF(\nabla_\pi Q(s_t,\pi(s_t;\theta);\omega))$$

用policy gradient descent 更新一次actor网络的值
$$\nabla_\theta \pi(s_t;\theta)\times meanOF(\nabla_\pi Q(s_t,\pi(s_t;\theta);\omega))$$



用soft update 更新target network的值<br>
$$\begin{gathered}
\omega^{\prime} \leftarrow \rho \omega^{\prime}+(1-\rho) \omega \\
\theta^{\prime} \leftarrow \rho \theta^{\prime}+(1-\rho) \theta
\end{gathered}$$

更新数轮后，再进行探索更新replay buffer，不断循环即可

# 代码部分

## 环境
tensorflow==2.3<br>
gym=0.23.1<br>
pandas=1.3.5<br>
numpy=0.18.5<br>
详见requirements.txt<br>
``` dos
pip list --format=freeze> requirements.txt
```

## import
引入<br>
os以实现通过--train和--test与--seed设置训练、测试和seed模式<br>
tensorflow实现神经网络的搭建<br>
time实现计时<br>
numpy实现部分数学运算与自由选择<br>
datetime实现对数据的管理<br>
matplotlib.pylot实现绘图需要<br>
gym引入环境<br>

In [1]:
# -*- coding: utf-8 _*_
# 系统级的函数
import os,time,datetime,argparse
# 尝试通过网上的方法屏蔽tensorflow的警告
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 基本配置
import numpy as np
import gym
import matplotlib.pyplot as plt

## tensorflow
import tensorflow as tf
# 一些层和模型的搭建
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.layers import Lambda as layer_Lambda
from tensorflow.keras.models import Model

# 自己编写的软更新函数
from class_ema import ema_V2


d:\New_Program_Files\anaconda_V5_3_1\envs\DDPG_V3\lib\site-packages\tensorflow\python\data\ops\iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
d:\New_Program_Files\anaconda_V5_3_1\envs\DDPG_V3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,
d:\New_Program_Files\anaconda_V5_3_1\envs\DDPG_V3\lib\site-packages\tensorflow\python\autograph\utils\testing.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
d:\New_Program_Files\anaconda_V5_3_1\envs\DDPG_V3\lib

## argparse setting
通过该项设置，实现键入**args，外部控制程序的能力，比如想要开始训练，在命令栏输入
``` dos
python DDPG.py --train
```

In [2]:
# parser = argparse.ArgumentParser(description='Train or test neural net motor controller.')
# parser.add_argument('--train', dest='train', action='store_true', default=False)    # 未键入--train时，args.train=Flase
# parser.add_argument('--test', dest='test', action='store_true') # 未键入--test时，args.test=Flase
# parser.add_argument('--seed',dest='seed',default=3047,type=int) # 未键入--seed 123 时，args.seed=3047
# args = parser.parse_args()  # 整合到args内

## 超参数
研究表明
critic的学习率最好是actor的数倍

In [6]:
ENV_NAME='Pendulum-v1'  # 环境
# RANDOMSEED=args.seed    # 随机种子
RANDOMSEED=3047   # 随机种子

LR_A=0.001              # actor learning rate
LR_C=0.002              # critic learning rate
GAMMA=0.9               # reward discount rate
TAU=0.001               # soft update coe
REPLAYBUFFER_SIZE=10000 # size of replay buffer
BATCH_SIZE=32           # learn per batch size
VAR=3                   # variance of the action for exploration

MAX_EPISODES=200        # maximum exploration times(from reset() to done/max_EP_STEP)
MAX_EP_STEPS=200        # maximum step per episode
TEST_PER_EPISODES=10    # step that test the model

In [9]:
class DDPG(object):
    
    '''
    replay_buffer

    '''

    def __init__(self,s_dim,a_dim,a_bound):
        # 先初始化transition (s_t,a_t,r_t,s_{t+1})
        self.replay_buffer=np.zeros(REPLAYBUFFER_SIZE,s_dim*2+a_dim*1+1,dtype=np.float32)
        self.pointer=0  # 指针
        self.s_dim,self.a_dim,self.a_bound=s_dim,a_dim,a_bound

        W_init = tf.random_normal_initializer(mean=0,stddev=0.3)
        b_init = tf.constant_initializer(0.1)

        # 建立四个网络，先定义两个网络架构
        def get_actor(input_state_shape,output_shape,name=''):
            lb=np.array(output_shape[0])
            ub=np.array(output_shape[1])
            gap=ub-lb
            '''
            input:s[n,s_dim]
            output:a[n,a_dim] in a_bound

            with a struct of
            input 64+dense 64+dense 1+tanh 1+(layer_Lambda)

            where n is the number of states input
            '''
            inputs=Input(input_state_shape,name=name+' input')
            x=Dense(units=64,activation='relu',kernel_initializer=W_init, bias_initializer=b_init,name=name+' invisiable layer1')(inputs)
            x=Dense(units=64,activation='relu',kernel_initializer=W_init, bias_initializer=b_init,name=name+' invisiable layer2')(x)
            x=Dense(units=1,activation='sigmoid',kernel_initializer=W_init, bias_initializer=b_init,name=name+' sigmoid layer')(x)
            x=layer_Lambda(lambda x:lb+gap*x,name=name+' Lambda layer')
            return Model(inputs=inputs,outputs=x,name=name+'network')
        
        def get_critic(input_s_shape,input_a_shape,name=''):
            '''
            input :s,a
            output:q_value
            with a struct of 
            [input_s,input_a]
            '''
            input_s=Input(input_s_shape,name=name+' state input')
            input_a=Input(input_a_shape,name=name+' action input')
            x=tf.concat([input_s,input_a],1)
            x=Dense(units=64,activation='relu',kernel_initializer=W_init, bias_initializer=b_init,name=name+' invisiable layer1')(x)
            x=Dense(units=64,activation='relu',kernel_initializer=W_init, bias_initializer=b_init,name=name+' invisiable layer2')(x)
            x=Dense(units=1,kernel_initializer=W_init, bias_initializer=b_init,name=name+' linear output')(x)
            return Model(inputs=[input_s,input_a],outputs=x,name=name+'network')
        
        # 帮助target网络硬更新
        def copy_para(from_model, to_model):
            '''
            copy the parameter from from_model to to_model
            '''
            for i,j in zip(from_model.trainable_weights,to_model.trainable_weights):
                j.assign(i)
        
        # 建立四个网络
        self.actor          =get_actor(s_dim,output_shape=a_bound,name='actor')
        self.actor_target   =get_actor(s_dim,output_shape=a_bound,name='actor_target')
        self.critic         =get_critic(s_dim,a_dim,name='critic')
        self.critic_target  =get_critic(s_dim,a_dim,name='critic_target')

        # 赋值
        copy_para(self.actor,self.actor_target)
        copy_para(self.critic,self.critic_target)

        # 初始化 ExponentialMovingAverage
        self.ema=ema_V2(decay=1-TAU)

        # 设置优化器
        self.actor_opt = tf.optimizers.Adam(LR_A)
        self.critic_opt = tf.optimizers.Adam(LR_C)

        ############*end* __init__ *end*############
    
    # 进行ema平滑更新函数
    def ema_update(self):
        '''
        shadow_variable -= (1 - decay) * (shadow_variable - variable)
        or shadow_variable = decay * shadow_variable + (1 - decay) * variable(not recommended)
        with decay=min(self.decay, (1 + num_updates) / (10 + num_updates))
        '''
        paras = self.actor.trainable_weights + self.critic.trainable_weights    #获取要更新的参数包括actor和critic的
        # self.ema.apply(paras)                                                   #主要是建立影子参数
        for i, j in zip(self.actor_target.trainable_weights + self.critic_target.trainable_weights, paras):
            self.ema.apply(i)                                                   #主要是建立影子参数
            i.assign(self.ema.average(j))  

    # 动作选择函数
    def choose_action(self,s):
        '''
        input: state s
        output: the action chosen by actor
        '''
        return self.actor(np.array(s, dtype=np.float32))# 限定输入
    
    # 存储transition
    def store_transition(self,s,a,r,s_):
        '''
        store transition(s,a,r,s') into the replay buffer
        '''
        s =s.astype(np.float32)
        s_ =s_.astype(np.float32)
        
        transition=np.hstack((s,a,r,s_))# 横向堆叠这些变量

        index = self.pointer % REPLAYBUFFER_SIZE
        self.replay_buffer[index,:]=transition
        self.pointer+=1

    # 学习一轮
    def learn(self):
        '''
        upgrade the parameter using TD_error and Policy Gradient Descent
        '''
        indices = np.random.choice(REPLAYBUFFER_SIZE,size=BATCH_SIZE) # 先确定一批transition的位置
        bt = self.memory[indices, :]                    #根据indices，选取数据bt，相当于随机
        bs = bt[:, :self.s_dim]                         #从bt获得数据s
        ba = bt[:, self.s_dim:self.s_dim + self.a_dim]  #从bt获得数据a
        br = bt[:, -self.s_dim - 1:-self.s_dim]         #从bt获得数据r
        bs_ = bt[:, -self.s_dim:]                       #从bt获得数据s'

        # 更新critic先
        with tf.GradientTape() as tape:         # 通过这种形式获得梯度
            a_ = self.actor_target(bs_)         
            q_ = self.critic_target([bs_, a_])
            y = br + GAMMA * q_
            q = self.critic([bs, ba])
            td_error = tf.losses.mean_squared_error(y, q) # TD_error MSE(y,q)
        c_grads = tape.gradient(td_error, self.critic.trainable_weights) # 得到偏LOSS偏omega
        self.critic_opt.apply_gradients(zip(c_grads, self.critic.trainable_weights))# 进行一次梯度下降（目的是降低LOSS）

        # 更新actor
        with tf.GradientTape() as tape:
            a = self.actor(bs)
            q = self.critic([bs, a])
            a_loss = -tf.reduce_mean(q)  # 【敲黑板】：注意这里用负号，是梯度上升！也就是离目标会越来越远的，就是越来越大。
        a_grads = tape.gradient(a_loss, self.actor.trainable_weights) # 得到*负值*偏Q偏theta
        self.actor_opt.apply_gradients(zip(a_grads, self.actor.trainable_weights))# 进行一次梯度下降（但是实际是梯度上升）

        # 更新一次参数
        self.ema_update()

    # 保存变量和load变量
    def save_ckpt(self):
        """
        save trained weights
        :return: None
        """
        if not os.path.exists('model'):
            os.makedirs('model')
        else:
            now=datetime.datetime.now()
            now=now.strftime('%Y-%m-%d-%H%M%S')
            new_name='model_'+now
            old_name='model'
            os.rename(old_name,new_name)
            os.makedirs(old_name)
            

        self.actor.save_weights('model/ddpg_actor.hdf5')
        self.actor_target.save_weights('model/ddpg_actor_target.hdf5')
        self.critic.save_weights('model/ddpg_critic.hdf5')
        self.critic_target.save_weights('model/ddpg_critic_target.hdf5')

    def load_ckpt(self):
        """
        load trained weights
        :return: None
        """
        self.actor.load_weights('model/ddpg_actor.hdf5')
        self.actor_target.load_weights('model/ddpg_actor_target.hdf5')
        self.critic.load_weights('model/ddpg_critic.hdf5')
        self.critic_target.load_weights('model/ddpg_critic_target.hdf5')


In [11]:
# init
env=gym.make(ENV_NAME)
env.unwrapped

# set seed 方便复现结果
env.reset(seed=RANDOMSEED)
np.random.seed(RANDOMSEED)
tf.random.set_seed(RANDOMSEED)

# 得到动作空间参数
s_dim = env.observation_space.shape[0]
a_dim = env.action_space.shape[0]
a_bound_lb = env.action_space.low
a_bound_ub = env.action_space.high
a_bound = [a_bound_lb,a_bound_ub]

# 初始化DDPG
ddpg=DDPG(s_dim,a_dim,a_bound)

# 开始训练 --train
if 1:
    # 部分信息输出
    print('s_dim',s_dim)
    print('a_dim',a_dim)
    print('a_bound',a_bound)
    print('\n**********',
    f'\n Random Seed is :\t {RANDOMSEED}',
    '\n**********')

    reward_buffer = []      #用于记录每个EP的reward，统计变化
    t0 = time.time()        #统计时间
    for i in range(MAX_EPISODES):
        t1 = time.time()
        s = env.reset()
        ep_reward = 0       #记录当前EP的reward
        for j in range(MAX_EP_STEPS):
            # Add exploration noise
            a = ddpg.choose_action(s)       #这里很简单，直接用actor估算出a动作

            # 为了能保持开发，这里用了另外一种方式增加探索。
            # 因此需要需要以a为均值，VAR为标准差，建立正态分布，再从正态分布采样出a
            # 因为a是均值，所以a的概率是最大的。但a相对其他概率由多大，是靠VAR调整。这里我们其实可以增加更新VAR，动态调整a的确定性
            # 然后进行裁剪
            a = np.clip(np.random.normal(a, VAR), -2, 2)  
            # 与环境进行互动
            s_, r, done, info = env.step(a)

            # 保存s，a，r，s_
            ddpg.store_transition(s, a, r / 10, s_)

            # 第一次数据满了，就可以开始学习
            if ddpg.pointer > REPLAYBUFFER_SIZE:
                ddpg.learn()

            #输出数据记录
            s = s_  
            ep_reward += r  #记录当前EP的总reward
            if j == MAX_EP_STEPS - 1:
                print(
                    '\rEpisode: {}/{}  | Episode Reward: {:.4f}  | Running Time: {:.4f}'.format(
                        i, MAX_EPISODES, ep_reward,
                        time.time() - t1
                    ), end=''
                )
            plt.show()
        # test
        if i and not i % TEST_PER_EPISODES:
            t1 = time.time()
            s = env.reset()
            ep_reward = 0
            for j in range(MAX_EP_STEPS):

                a = ddpg.choose_action(s)  # 注意，在测试的时候，我们就不需要用正态分布了，直接一个a就可以了。
                s_, r, done, info = env.step(a)

                s = s_
                ep_reward += r
                if j == MAX_EP_STEPS - 1:
                    print(
                        '\rEpisode: {}/{}  | Episode Reward: {:.4f}  | Running Time: {:.4f}'.format(
                            i, MAX_EPISODES, ep_reward,
                            time.time() - t1
                        )
                    )

                    reward_buffer.append(ep_reward)

        if reward_buffer:
            plt.ion()
            plt.cla()
            plt.title('DDPG')
            plt.plot(np.array(range(len(reward_buffer))) * TEST_PER_EPISODES, reward_buffer)  # plot the episode vt
            plt.xlabel('episode steps')
            plt.ylabel('normalized state-action value')
            plt.ylim(-2000, 100)
            plt.show()
            plt.pause(0.1)
    print('\n***end***')
    plt.ioff()
    # plt.show()      
    print('\nRunning time: ', time.time() - t0)
    # time_start_store_weights=time.perf_counter()
    # print('store_time')
    ddpg.save_ckpt()
    plt.savefig('model/result.png')
    # print('store time cost is:{:.4f}'.format(time.perf_counter()-time_start_store_weights))
    

# test
if 2:
    ddpg.load_ckpt()
    while True:
        s = env.reset()
        for i in range(MAX_EP_STEPS):
            env.render()
            s, r, done, info = env.step(ddpg.choose_action(s))
            if done:
                break






TypeError: data type not understood